In [10]:
from pyspark import SparkContext

In [11]:
sc = SparkContext.getOrCreate()

In [12]:
rdd=sc.textFile("datasets/DolarVerileri.txt")
rdd = rdd.map(lambda line : line.replace(".","").replace(",","."))
rdd = rdd.filter(lambda line : len(line.split("\t")[2]) > 0)


In [13]:
def splitLine(line):
    x = line.split("\t")
    return (x[1],float(x[2]))

In [14]:
rdd = rdd.map(lambda line : splitLine(line) )
rdd1 = rdd.zipWithIndex().map(lambda x : (x[1],x[0]))
rdd2 = rdd1.map(lambda x: (x[0]+1,x[1]))
joinedRDD = rdd1.join(rdd2).sortBy(lambda x: x[0])
joinedRDD.take(5)

[(1, (('03-01-1950', 2.8), ('02-01-1950', 2.8))),
 (2, (('04-01-1950', 2.8), ('03-01-1950', 2.8))),
 (3, (('05-01-1950', 2.8), ('04-01-1950', 2.8))),
 (4, (('06-01-1950', 2.8), ('05-01-1950', 2.8))),
 (5, (('09-01-1950', 2.8), ('06-01-1950', 2.8)))]

In [15]:
def calculateDiffAndPer(item):
    prevDay = item[1][0][1]
    curDay = item[1][1][1]
    dates = item[1][0][0]+"-"+item[1][1][0]
    percentage = (curDay-prevDay)/prevDay
    return (dates,percentage)

### We need to remove the end of the year 2004 when 6 zeros were droped from TL

## The mosd depericiated day of Turkish Lira
*Except 31.12.2004 it is the day when 6 zeros were droped*

In [16]:
joinedRDD=joinedRDD.map(lambda x : calculateDiffAndPer(x))
joinedRDD=joinedRDD.sortBy(lambda x :x[1],ascending=False)


joinedRDD = joinedRDD.filter(lambda x: x[0] != '03-01-2005-31-12-2004')

print(joinedRDD.take(5))

[('03-01-2005-31-12-2004', 1001566.1641791044), ('27-02-2001-26-02-2001', 0.13387001667536716), ('31-10-2008-30-10-2008', 0.12666666666666662), ('25-03-1991-22-03-1991', 0.122293930234458), ('11-04-1994-08-04-1994', 0.09863817067204128)]
